### This dataset consists of competitors at the Rio Olympics in 2016.

### There are a few possibilities of predictions that can be made using this dataset.  Our first run through will be looking to classify the athlete's country using recipients that earned a medal.  What features may be of importance?

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('athletes.csv')

In [3]:
df.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [4]:
df.shape

(11538, 11)

In [12]:
df.isnull().sum()

id               0
name             0
nationality      0
sex              0
dob              1
height         330
weight         659
sport            0
gold             0
silver           0
bronze           0
dtype: int64

### There are nulls, but not a lot.  Focusing on time, let's drop them.

In [13]:
df.dropna(inplace=True)

In [15]:
df.isnull().sum()

id             0
name           0
nationality    0
sex            0
dob            0
height         0
weight         0
sport          0
gold           0
silver         0
bronze         0
dtype: int64

In [16]:
df.shape

(10858, 11)

In [18]:
df.dtypes

id               int64
name            object
nationality     object
sex             object
dob             object
height         float64
weight         float64
sport           object
gold             int64
silver           int64
bronze           int64
dtype: object

In [28]:
df[(df.gold == 1) | (df.silver == 1) | (df.bronze == 1)].shape

(1704, 11)

### This is good to see.  Looking at athletes that do have a medal still leaves us at 1,704 observations. 